In [21]:
import pandas as pd


# Answers to the questions can be placed in Jupyter Notebook cells of type markdown.

# - Is there a relationship between the number of children and timely loan repayment?  
# - Is there a relationship between marital status and timely loan repayment?  
# - Is there a relationship between income level and timely loan repayment?  
# - How do different loan purposes affect timely loan repayment?

In [2]:
df = pd.read_csv('./datasets/loan_reliability_project.csv')
print(df.head(5))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жи

In [3]:
# There was more than 2000 null values in 'days_employed' column which is more than 10% of total values.
median_days_employed = df.loc[df['days_employed'] > 0, 'days_employed'].median()
# I changed missed values to median value because it handles outliers well,
# keeps the data balanced, and ensures the analysis stays accurate
df['days_employed'] = df['days_employed'].fillna(value=median_days_employed)

In [4]:
# Negative values (18080) in 'days_employed' column seems incorrect and I changed them to median too
df.loc[df['days_employed'] <= 0, 'days_employed'] = median_days_employed

# Checking that all the values are grater than 0
print(df.loc[df['days_employed'] <= 0, 'days_employed'].sum())

0.0


In [5]:
# Making sure the type is correct
print(df['days_employed'].dtype, df['dob_years'].dtype, df['total_income'].dtype)
#float64 int64 float64

float64 int64 float64


In [6]:
# It's more comfortable to work with int numbers in salary column
df['total_income'] = df['total_income'].fillna(0).astype('int')
print(df['total_income'].dtype)

int64


In [7]:
# Checking for duplicates in columns education, family_status, income_type, purpose

print(f'education: {df['education'].unique()}')
print(f'family_status: {df['family_status'].unique()}')
print(f'income_type: {df['income_type'].unique()}')
print(f'purpose: {df['purpose'].unique()}')

# There are duplicates in education and purpose columns

education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимость

In [8]:
# To lowcase
df.loc[:, 'education'] = df.loc[:, 'education'].str.lower()

print(f'education: {df['education'].unique()}')

education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [9]:
# I'd like to create a new column to group the data from 'purpose' column

def categorize_purpose(purpose):
    purpose = purpose.lower().strip()
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образован' in purpose:
        return 'получение образования'
    else:
        return 'прочее'

df['purpose_categories'] = df['purpose'].apply(categorize_purpose)

print(f'purpose_categories: {df['purpose_categories'].unique()}')

# Check if all categories are collected
if df.loc[df['purpose_categories'] == 'прочее'].shape[0] == 0:
    print('All the purposes are categorized')
else:
    print(df.loc[df['purpose_categories'] == 'прочее'])

purpose_categories: ['операции с недвижимостью' 'операции с автомобилем'
 'получение образования' 'проведение свадьбы']
All the purposes are categorized


In [10]:
print(df.head(5))

   children  days_employed  dob_years education  education_id  \
0         1  365213.306266         42    высшее             0   
1         1  365213.306266         36   среднее             1   
2         0  365213.306266         33   среднее             1   
3         3  365213.306266         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose        purpose_categories  
0               покупка жилья  операции с недвижимостью  
1     приобр

In [11]:
# Removing extra data from the df
education_df = df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
family_status_df = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
df.drop(columns=['education', 'family_status'], inplace=True)
print(education_df)
print(family_status_df)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


In [12]:
print(df.head(5))

   children  days_employed  dob_years  education_id  family_status_id gender  \
0         1  365213.306266         42             0                 0      F   
1         1  365213.306266         36             1                 0      F   
2         0  365213.306266         33             1                 0      M   
3         3  365213.306266         32             1                 0      M   
4         0  340266.072047         53             1                 1      F   

  income_type  debt  total_income                     purpose  \
0   сотрудник     0        253875               покупка жилья   
1   сотрудник     0        112080     приобретение автомобиля   
2   сотрудник     0        145885               покупка жилья   
3   сотрудник     0        267628  дополнительное образование   
4   пенсионер     0        158616             сыграть свадьбу   

         purpose_categories  
0  операции с недвижимостью  
1    операции с автомобилем  
2  операции с недвижимостью  
3     по

In [13]:
# Categorising income

def categorising_income(income):
    if income <= 30000:
        return 'E'
    elif income > 30000 & income <= 50000:
        return 'D'
    elif income > 50000 & income <= 200000:
        return 'C'
    elif income > 200000 & income <= 1000000:
        return 'B'
    elif income > 1000000:
        return 'A'
    else:
        return 'Income error'

df['total_income_category'] = df['total_income'].apply(categorising_income)
print(df['total_income_category'].unique())

['D' 'E']


In [14]:
data_pivot = df.pivot_table(
    index='family_status_id',  # Family
    columns='children',     # Kids
    values='debt',          # Debts
    aggfunc='sum'           # Sum
)
print(data_pivot)

children          -1      0      1      2     3    4    5    20
family_status_id                                               
0                 1.0  516.0  246.0  145.0  17.0  3.0  0.0  3.0
1                 0.0  229.0  118.0   30.0   8.0  0.0  0.0  3.0
2                 0.0   53.0    7.0    3.0   0.0  0.0  NaN  0.0
3                 0.0   55.0   21.0    7.0   1.0  0.0  NaN  1.0
4                 0.0  210.0   52.0    9.0   1.0  1.0  NaN  1.0


In [15]:
print(df)

       children  days_employed  dob_years  education_id  family_status_id  \
0             1  365213.306266         42             0                 0   
1             1  365213.306266         36             1                 0   
2             0  365213.306266         33             1                 0   
3             3  365213.306266         32             1                 0   
4             0  340266.072047         53             1                 1   
...         ...            ...        ...           ...               ...   
21520         1  365213.306266         43             1                 1   
21521         0  343937.404131         67             1                 0   
21522         1  365213.306266         38             1                 1   
21523         3  365213.306266         38             1                 0   
21524         2  365213.306266         40             1                 0   

      gender income_type  debt  total_income                       purpose 

In [16]:
# Removing NaN
df['debt'] = df['debt'].fillna(0)

# Checking num of children
print(df['children'].value_counts())

# A value of -1 for children is impossible and it will be replaced with 0
df.loc[df['children'] == -1, 'children'] = 0

# The value of 20 children seems highly unlikely
median_children = df['children'].median()
df.loc[df['children'] == 20, 'children'] = median_children

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: count, dtype: int64


In [22]:
data_pivot_family = df.pivot_table(
    index='family_status_id',  
    columns='children',                             
    values='debt',                                  
    aggfunc='sum'                                   
)

print(data_pivot_family)

children              0      1      2     3    4    5
family_status_id                                     
0                 520.0  246.0  145.0  17.0  3.0  0.0
1                 232.0  118.0   30.0   8.0  0.0  0.0
2                  53.0    7.0    3.0   0.0  0.0  NaN
3                  56.0   21.0    7.0   1.0  0.0  NaN
4                 211.0   52.0    9.0   1.0  1.0  NaN


In [25]:
data_pivot_income = df.pivot_table(
    index='income_type',  
    columns='total_income_category',                            
    values='debt',                                                                     
    aggfunc='sum'                                   
)

print(data_pivot_income)

total_income_category      D     E
income_type                       
безработный              1.0   NaN
в декрете                1.0   NaN
госслужащий             79.0   7.0
компаньон              346.0  30.0
пенсионер              179.0  37.0
предприниматель          0.0   0.0
сотрудник              963.0  98.0
студент                  0.0   NaN


In [35]:
data_debt_purposes = df.groupby(['purpose_categories','income_type'])['debt'].sum()                            

print(data_debt_purposes)

purpose_categories        income_type    
операции с автомобилем    в декрете            1
                          госслужащий         22
                          компаньон           85
                          пенсионер           51
                          сотрудник          244
операции с недвижимостью  безработный          1
                          госслужащий         36
                          компаньон          167
                          пенсионер           92
                          предприниматель      0
                          сотрудник          486
                          студент              0
получение образования     госслужащий         21
                          компаньон           72
                          пенсионер           48
                          сотрудник          229
проведение свадьбы        госслужащий          7
                          компаньон           52
                          пенсионер           25
                          п

In [37]:
# the total number of credits for each purpose 
total_credits = df.groupby('purpose_categories')['debt'].count()

# total number of debts
total_debts = df.groupby('purpose_categories')['debt'].sum()

# debt ratio
debt_ratio = (total_debts / total_credits) * 100

# create a DataFrame to summarize the analysis
debt_analysis = pd.DataFrame({
    'Total Credits': total_credits,  # Total number of credits
    'Total Debts': total_debts,      # Total number of debts
    'Debt Ratio (%)': debt_ratio     # Percentage of credits with debts
}).sort_values(by='Debt Ratio (%)', ascending=False)

print(debt_analysis)

                          Total Credits  Total Debts  Debt Ratio (%)
purpose_categories                                                  
операции с автомобилем             4315          403        9.339513
получение образования              4022          370        9.199403
проведение свадьбы                 2348          186        7.921635
операции с недвижимостью          10840          782        7.214022
